### Libraries

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# (Optional) If you're working inside Jupyter
%load_ext autoreload
%autoreload 2
%matplotlib inline



In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
from data_processor_v2 import DataProcessorUpdated

processor = DataProcessorUpdated()

In [ ]:
processor.load_and_clean_data()

In [ ]:
processor.combine_all_data()

In [ ]:
processor.df

In [ ]:
processor.shift_dap()

In [ ]:
processor.df

In [ ]:
processor.split_data(feature_columns=['DAP_SystemLambda', 'SCED_system_lambda','Fuel_solar','Fuel_wind','Load_load'])

In [ ]:
processor.standardize_data()

In [ ]:
processor.shift_data()

In [ ]:
df, x_train_lstm, y_train_lstm, x_val_lstm, y_val_lstm, x_test_lstm, y_test_lstm = processor.get_data()

In [ ]:
x_train_lstm.shape

In [ ]:
y_val_lstm = y_val_lstm.squeeze(-1)
y_test_lstm = y_test_lstm.squeeze(-1)
y_train_lstm = y_train_lstm.squeeze(-1)


In [ ]:
print("x_train:", x_train_lstm.shape)   # 期望 (N, 32, 2) 或 (N, 24, 2)
print("y_train:", y_train_lstm.shape)   # 期望 (N, 32)   或 (N, 24)


In [ ]:
x_test_lstm.shape

# Model Building

In [80]:
import torch
from CNN_MLP import CNN2D_MLP_Model   # 根据你的文件结构保持不变

# 1. 选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 实例化模型（符合新签名）
cnn_mlp = CNN2D_MLP_Model(
    n_features=5,            # 输入特征数
    horizon=24,              # 预测步长
    cnn_channels=64,         # 卷积通道
    hidden_dims=[256,512,256]  # MLP 隐层
).to(device)

In [81]:
cnn_mlp.eval()
with torch.no_grad():
    y_pred = cnn_mlp(torch.tensor(x_test_lstm, dtype=torch.float32).to(device))


y_pred = y_pred.cpu().numpy()
errors = y_pred - y_test_lstm  # assuming y_test is numpy array
mse = np.mean(errors**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(errors))

print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")


Test MAE: 0.1508
Test RMSE: 0.8769


In [82]:
from ModelTrainer import ModelTrainer
from losses import fluctuation_loss  # or define it above in your notebook

trainer = ModelTrainer(
    model=cnn_mlp,
    features_training_data=x_train_lstm,
    target_training_data=y_train_lstm,
    features_eval_data=x_val_lstm,
    target_eval_data=y_val_lstm,
    device=device,
    loss_fn=lambda pred, target: fluctuation_loss(pred, target, alpha=0.11866313536860208)
)

trainer.train(epochs=100, batch_size=32, patience=50, learning_rate=1.0199709447160282e-05)

Epoch 1: Train Loss = 1.0320, Eval Loss = 0.2533
Epoch 2: Train Loss = 0.9872, Eval Loss = 0.2544
Epoch 3: Train Loss = 0.9711, Eval Loss = 0.2535
Epoch 4: Train Loss = 0.9780, Eval Loss = 0.2492
Epoch 5: Train Loss = 0.9455, Eval Loss = 0.2526
Epoch 6: Train Loss = 0.9336, Eval Loss = 0.2524
Epoch 7: Train Loss = 0.9229, Eval Loss = 0.2608
Epoch 8: Train Loss = 0.9103, Eval Loss = 0.2593
Epoch 9: Train Loss = 0.8996, Eval Loss = 0.2576
Epoch 10: Train Loss = 0.8893, Eval Loss = 0.2631
Epoch 11: Train Loss = 0.8776, Eval Loss = 0.2639
Epoch 12: Train Loss = 0.8692, Eval Loss = 0.2695
Epoch 13: Train Loss = 0.8610, Eval Loss = 0.2695
Epoch 14: Train Loss = 0.8547, Eval Loss = 0.2754
Epoch 15: Train Loss = 0.8443, Eval Loss = 0.2845
Epoch 16: Train Loss = 0.8379, Eval Loss = 0.2956
Epoch 17: Train Loss = 0.8291, Eval Loss = 0.2937
Epoch 18: Train Loss = 0.8256, Eval Loss = 0.2930
Epoch 19: Train Loss = 0.8201, Eval Loss = 0.2904
Epoch 20: Train Loss = 0.8188, Eval Loss = 0.3144
Epoch 21:

In [84]:
cnn_mlp.eval()
with torch.no_grad():
    y_pred = cnn_mlp(torch.tensor(x_test_lstm, dtype=torch.float32).to(device))


y_pred = y_pred.cpu().numpy()
errors = y_pred - y_test_lstm  # assuming y_test is numpy array
mse = np.mean(errors**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(errors))

# 1. Flatten both (remove batch and window dimensions if needed)
y_pred_flat = y_pred.reshape(-1)
y_test_flat = y_test_lstm.reshape(-1)

# 2. Denormalize both using your processor's function
y_pred_real = processor.denormalization(y_pred_flat)
y_test_real = processor.denormalization(y_test_flat)

from sklearn.metrics import mean_absolute_error
import numpy as np

mae = mean_absolute_error(y_test_real, y_pred_real)
mse = np.mean((y_test_real - y_pred_real) ** 2)
variance = np.var(y_test_real)
smse = mse / variance

print(f"MAE: {mae:.4f}, SMSE: {smse:.4f}")

MAE: 44.5838, SMSE: 1.0093


In [ ]:
loss = trainer.history
with open('CNN_MLP_Best.csv', 'w') as f:
    pd.DataFrame(loss).to_csv(f, index=False)

In [ ]:
import optuna
import torch
from CNN_MLP import CNN2D_MLP_Model
from ModelTrainer import ModelTrainer
from losses import fluctuation_loss



# ------------- 设备 -------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------- Optuna 目标函数 -------------------
def objective(trial):
    # ---- 超参数搜索空间 ----
    cnn_channels = trial.suggest_categorical("cnn_channels", [32, 64, 128])
    hidden_dim   = trial.suggest_categorical("mlp_hidden_dim", [128, 256, 512])
    n_layers     = trial.suggest_int("n_layers", 2, 4)
    lr           = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    alpha        = trial.suggest_uniform("alpha", 0.1, 0.5)

    hidden_dims = [hidden_dim] * n_layers   # MLP 隐层列表

    # ---- 构建模型 ----
    model = CNN2D_MLP_Model(
        n_features=5,        # 输入特征数
        horizon=24,          # 预测步长
        cnn_channels=cnn_channels,
        hidden_dims=hidden_dims
    ).to(device)

    # ---- 训练器 ----
    trainer = ModelTrainer(
        model=model,
        features_training_data=x_train_lstm,
        target_training_data=y_train_lstm,
        features_eval_data=x_val_lstm,
        target_eval_data=y_val_lstm,
        device=device,
        loss_fn=lambda pred, target: fluctuation_loss(pred, target, alpha=alpha)
    )

    # ---- 训练，控制轮数别太大避免单次 trial 过慢 ----
    trainer.train(epochs=50, batch_size=32, patience=10, learning_rate=lr)

    # ---- 返回验证集最后一次 (或最优) 损失 ----
    return trainer.history['eval_loss'][-1]

# ------------- 创建并运行 Optuna Study -------------------
study = optuna.create_study(direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=50)

print("最佳 trial:", study.best_trial.params)
print("最佳 eval_loss:", study.best_value)


In [ ]:

study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.HyperbandPruner()
)
study.optimize(objective, n_trials=50)


In [ ]:
print("Best hyperparameters:", study.best_trial.params)


In [ ]:
# predict

mlp_model.eval()
with torch.no_grad():
    y_pred = mlp_model(torch.tensor(x_test_mlp, dtype=torch.float32).to(device))


y_pred = y_pred.cpu().numpy()
errors = y_pred - y_test_mlp  # assuming y_test is numpy array
mse = np.mean(errors**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(errors))

print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")

